## Make a simple synthetic dataset to test hierarchical merge in FOF algorithm

#### the idea is this: 

* after the local FOF stage, each partition reports the particles it holds in the overlap region
* do a reduceByKey or treeAggregate of some sort to collect the groups belonging to the same particles
* produce a mapping of $G -> G_1$ and distribute to all hosts in form of broadcast lookup table

In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

import sys
sys.setrecursionlimit(sys.getrecursionlimit()*10)
import matplotlib.patches as patches

plt.style.use('bmh')

In [2]:
import findspark
findspark.init()

In [3]:
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))
    plt.draw()
    plt.show()

## Start Spark

In [8]:
import findspark
findspark.init()

In [9]:
import os
os.environ['SPARK_CONF_DIR'] = './conf'
os.environ['SPARK_DRIVER_MEMORY'] = '8G'

In [10]:
import pyspark
from pyspark import SparkContext, SparkConf
import pynbody

In [11]:
conf = SparkConf()

In [12]:
conf.set('spark.python.profile', 'true')
conf.set('spark.executor.memory', '8G')
conf.set('spark.driver.memory', '8G')
conf.set('spark.driver.maxResultSize', '5G')

In [13]:
import subprocess, re

In [14]:
from IPython.display import HTML

out = subprocess.check_output(["bjobs","-o","job_name jobid"])
jobid = re.findall('spark-fof (\d+)', out)[0]

job_peek = subprocess.check_output(["bpeek", str(jobid)])

master_url, master_webui = re.findall('(spark://\S+:\d{4}|http://\S+:\d{4})', job_peek)

HTML("""<p>master running at %s</p>
     <p>Web UI available <a target='_blank' href='%s\'>here</a>
     """%(master_url,master_webui))

In [15]:
sc = SparkContext(master=master_url, conf=conf)

In [16]:
# sc.addPyFile('spark_fof.py')
# # sc.addPyFile('spark_util.py')
# sc.addPyFile('spark_fof_c.pyx')
# sc.addPyFile('spark_fof_c.c')
# sc.addPyFile('spark_fof_c.so')
# sc.addPyFile('fof.so')


In [17]:
import spark_fof

### Make the base RDD

In [18]:
def convert_to_fof_particle(s): 
    p_arr = np.frombuffer(s, pdt_tipsy)
    
    new_arr = np.zeros(len(p_arr), dtype=pdt)
    new_arr['pos'] = p_arr['pos']
    
    return new_arr

In [19]:
def get_minmax(index, i): 
    mins = []
    maxs = []
    for arr in i: 
        mins.append(arr['pos'].min(axis=0))
        maxs.append(arr['pos'].max(axis=0))
    
    yield (index, (np.concatenate(mins).reshape(len(mins),3).min(axis=0), np.concatenate(maxs).reshape(len(mins),3).max(axis=0)))

In [20]:
from pyspark.accumulators import AccumulatorParam

class dictAdd(AccumulatorParam):
    def zero(self, value):
        return {i:0 for i in range(len(value))}
    def addInPlace(self, val1, val2): 
        for k, v in val2.iteritems(): 
            val1[k] += v
        return val1

In [21]:
# function to map from file block numbers to domain bin
N = 62
map_file_to_domain = lambda (x,y,z): (x-1) + (y-1)*N + (z-1)*N*N

pdt_lc = np.dtype([('pos', 'f4', 3),('vel', 'f4', 3)])

import re

get_block_ids = re.compile('blk\.(\d+)\.(\d+)\.(\d+)i')

def make_lc_rdd(sc, path):
    from glob import glob

    def set_particle_IDs_partition(index, iterator): 
        p_counts = partition_counts.value
        local_index = 0
        start_index = sum([p_counts[i] for i in range(index)])
        for arr in iterator:
            arr['iOrder'] = range(start_index + local_index, start_index + local_index + len(arr))
            local_index += len(arr)
            yield arr
    
    def read_file(index, i, chunksize=102400): 
        for f in i:
            with open(f,'rb') as f: 
                header = f.read(62500)
                while True:
                    chunk = f.read(chunksize*24)
                    if len(chunk): 
                        p_arr = np.frombuffer(chunk, pdt_lc)
                        new_arr = np.zeros(len(p_arr), dtype=pdt)
                        new_arr['pos'] = p_arr['pos']
                        npart_acc.add({index: len(p_arr)})
                        yield new_arr
                    else: 
                        break
                    
    files = glob('/cluster/home/roskarr/projects/euclid/2Tlc-final/*/*')

    ids = map(lambda x: tuple(map(int, get_block_ids.findall(x)[0])), files)
    ids_map = {x:i for i,x in enumerate(ids)}
    ids_map_b = sc.broadcast(ids_map)
    nfiles = len(files)

    print 'Number of input files: ', nfiles
    
    # set the partition count accumulator
    npart_acc = sc.accumulator({i:0 for i in range(nfiles)}, dictAdd())
    
    rec_rdd = (sc.parallelize(zip(ids,files), numSlices=27).partitionBy(nfiles, lambda x: ids_map_b.value[x])
                 .values()
                 .mapPartitionsWithIndex(read_file)).cache()
    
    rec_rdd.count()
    partition_counts = sc.broadcast(npart_acc.value)
    
    return partition_counts, ids_map, rec_rdd.mapPartitionsWithIndex(set_particle_IDs_partition)

In [22]:
%%time 
p_counts, ids_map, p_rdd = make_lc_rdd(sc, '2Tlc-final/')
p_rdd.cache()
#p_rdd.count()

Number of input files:  27
CPU times: user 40 ms, sys: 10 ms, total: 50 ms
Wall time: 1min 21s


In [23]:
%time limits = p_rdd.mapPartitionsWithIndex(get_minmax).collect()

CPU times: user 20 ms, sys: 10 ms, total: 30 ms
Wall time: 1min


In [24]:
diff = (limits[0][1][1] - limits[0][1][0])[0]
global_min = -31*diff
global_max = 31*diff

diff, global_min, global_max

(0.03306878, -1.0251321755349636, 1.0251321755349636)

In [25]:
dom_maxs = np.array([global_max]*3, dtype=np.float64)
dom_mins = np.array([global_min]*3, dtype=np.float64)

In [26]:
reload(spark_fof.spark_fof)

<module 'spark_fof.spark_fof' from '/cluster/project/sis/ri/roskarr/spark-fof/spark_fof/spark_fof.pyc'>

In [27]:
#tau = diff*5./125.
tau = 0.2/12600
domain_containers = spark_fof.spark_fof.setup_domain(62, tau, dom_mins, dom_maxs)

In [28]:
global_to_cutout_map = {}
for k,v in ids_map.iteritems(): 
    global_to_cutout_map[map_file_to_domain(k)] = v

### Run FOF and Merge groups

In [29]:
n_containers = len(domain_containers)

In [30]:
mins = np.zeros((n_containers, 3))
maxs = np.zeros((n_containers, 3))
mins_buff = np.zeros((n_containers, 3))
maxs_buff = np.zeros((n_containers, 3))

In [31]:
for i in range(n_containers):
    r = domain_containers[i]
    mins[i] = r.mins
    maxs[i] = r.maxes
    mins_buff[i] = r.bufferRectangle.mins
    maxs_buff[i] = r.bufferRectangle.maxes

In [34]:
fof_analyzer = spark_fof.spark_fof.FOFAnalyzer(sc, p_rdd, 1, 62, tau, dom_mins, dom_maxs, Npartitions=27)

In [35]:
### THIS MUST BE FIXED IN THE MAIN CODE
fof_analyzer.global_to_local_map = global_to_cutout_map

In [37]:
fof_rdd = fof_analyzer.run_fof()

In [38]:
fof_rdd.count()

7538

In [40]:
fof_rdd.first()

array([ ([0.00017607540939934552, -0.06500079482793808, -0.06604167073965073], 0, 0, 201),
       ([0.0001809162349672988, -0.06499974429607391, -0.06601879745721817], 0, 1, 201),
       ([0.0001694740931270644, -0.06500272452831268, -0.06602748483419418], 0, 2, 201),
       ...,
       ([0.00039089034544304013, -0.06376203894615173, -0.046201929450035095], 0, 262141, 3783651),
       ([0.0003938704030588269, -0.06377732008695602, -0.04616163671016693], 0, 262142, 3783639),
       ([0.00037741009145975113, -0.06378830969333649, -0.04617900401353836], 0, 262143, 3790729)], 
      dtype={'names':['pos','is_ghost','iOrder','iGroup'], 'formats':[('<f4', (3,)),'<i4','<i4','<i8'], 'offsets':[0,12,16,24], 'itemsize':32})

In [41]:
merged_rdd = fof_analyzer.merge_groups(0)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 5 cancelled because Stage 14 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1454)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1393)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1381)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1380)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:234)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:138

In [ ]:
merged = merged_rdd.collect()

In [ ]:
merged_arr = np.fromiter(merged, pdt)

In [ ]:
groups = np.unique(merged_arr['iGroup'])

In [ ]:
final_fof_rdd = fof_analyzer.final_fof_rdd

In [ ]:
len(groups)

In [ ]:
merged_arr['iGroup'].max()

In [ ]:
particles = np.fromiter(ps,pdt)

In [ ]:
ps_0 = np.fromiter(part_rdd.glom().collect()[0], pdt)

In [ ]:
fof.run(particles, tau, 1)

In [ ]:
fof.run(ps_0, tau, 1)

In [ ]:
f, axs = plt.subplots(1,2, subplot_kw={'aspect':'equal'}, figsize=(12,12))
for group in groups: 
    inds = np.where(merged_arr['iGroup'] == group)[0]
    print group, len(inds)
    axs[0].plot(merged_arr['pos'][inds,1], merged_arr['pos'][inds,0], ',')
    axs[1].plot(merged_arr['pos'][inds,2], merged_arr['pos'][inds,0], ',')
for ax in axs:
    ax.set_xlim(-1,1); ax.set_ylim(-1,1)

In [ ]:
sc.show_profiles()

In [ ]:
%time fof.run(particles, tau)

## Used for testing

In [ ]:
def read_tipsy_output(filename, chunksize = 2048): 
    """
    Read a tipsy file and set the sequential particle IDs
    
    This scans through the data twice -- first to get partition particle counts
    and a second time to actually set the particle IDs.
    """
    
    # helper functions
    def convert_to_fof_particle_partition(index, iterator): 
        for s in iterator: 
            a = convert_to_fof_particle(s)
            if count: 
                npart_acc.add({index: len(a)})
            yield a

    def set_particle_IDs_partition(index, iterator): 
        p_counts = partition_counts.value
        local_index = 0
        start_index = sum([p_counts[i] for i in range(index)])
        for arr in iterator:
            arr['iOrder'] = range(start_index + local_index, start_index + local_index + len(arr))
            local_index += len(arr)
            yield arr
    
    rec_rdd = sc.binaryRecords(filename, pdt_tipsy.itemsize*chunksize)
    nPartitions = rec_rdd.getNumPartitions()
    # set the partition count accumulator
    npart_acc = sc.accumulator({i:0 for i in range(nPartitions)}, dictAdd())
    count=True
    # read the data and count the particles per partition
    rec_rdd = rec_rdd.mapPartitionsWithIndex(convert_to_fof_particle_partition)
    rec_rdd.count()
    count=False

    partition_counts = sc.broadcast(npart_acc.value)

    return rec_rdd.mapPartitionsWithIndex(set_particle_IDs_partition)

In [ ]:
#p_rdd = read_tipsy_output('/Users/rok/polybox/euclid256.nat_no_header')

In [ ]:
#p_rdd.cache().count()

In [ ]:
# %%time 
# nMinMembers = 1
# n_groups = fof.run(ps, tau, nMinMembers)
# print 'number of groups to %d particle = %d'%(nMinMembers, n_groups)